# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json

# Import API key
from api_keys import weather_api_key
from api_keys import g_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=100)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=100)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

82

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
# Url and units
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metrics"

# Build query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="
print(query_url)

http://api.openweathermap.org/data/2.5/weather?appid=932311d6e2f00289b7cc4ad01edf2b31&units=metrics&q=


In [5]:
# Loop through the list of cities and request for data on each
Lat = []
Lng = []
Max_Temp = []
Humidity = []
Cloudiness = []
Wind_Speed = []
Country = []
Date = []

print("Beginning Data Retrieval")
print("--------------------------------------------")
record = 0
set = 1
for index, city in enumerate(cities):
    record = record + 1
    # To avoid api call rate limits, get city weather data for 50 cities,
    # then sleep for 5 seconds, and then continue with next 50 cities and so on...
    if record == 51:
        record = 1
        set = set + 1
        time.sleep(5)
    print(f"Processing Record {record} of Set {set} | {city}")
    try:
        response = requests.get(query_url + city).json()
        Lat.append(response['coord']['lat'])
        Lng.append(response['coord']['lon'])
        Max_Temp.append(response['main']['temp_max'])
        Humidity.append(response['main']['humidity'])
        Wind_Speed.append(response['wind']['speed'])
        Date.append(response['dt'])
        Country.append(response['sys']['country'])
        Cloudiness.append(response['clouds']['all'])
    except KeyError:
        print("City not found. Skipping...")
        Lat.append(np.nan)
        Lng.append(np.nan)
        Max_Temp.append(np.nan)
        Humidity.append(np.nan)
        Wind_Speed.append(np.nan)
        Date.append(np.nan)
        Country.append(np.nan)
        Cloudiness.append(np.nan)

    if set == 4:
        break
print("------------------------------------------------")
print("Data Retrieval Complete")
print("------------------------------------------------")

Beginning Data Retrieval
--------------------------------------------
Processing Record 1 of Set 1 | east london
Processing Record 2 of Set 1 | hilo
Processing Record 3 of Set 1 | ribeira grande
Processing Record 4 of Set 1 | rikitea
Processing Record 5 of Set 1 | punta arenas
Processing Record 6 of Set 1 | saint-philippe
Processing Record 7 of Set 1 | port-gentil
Processing Record 8 of Set 1 | pacifica
Processing Record 9 of Set 1 | kangaatsiaq
Processing Record 10 of Set 1 | ushuaia
Processing Record 11 of Set 1 | castro
Processing Record 12 of Set 1 | pangnirtung
Processing Record 13 of Set 1 | busselton
Processing Record 14 of Set 1 | izhmorskiy
Processing Record 15 of Set 1 | vaitupu
City not found. Skipping...
Processing Record 16 of Set 1 | hobart
Processing Record 17 of Set 1 | provideniya
Processing Record 18 of Set 1 | bandarbeyla
Processing Record 19 of Set 1 | hermanus
Processing Record 20 of Set 1 | lebu
Processing Record 21 of Set 1 | deputatskiy
Processing Record 22 of S

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [13]:
#Creating a dataframe to fill after looping in cities
cities_df = pd.DataFrame({
    'City': cities,
    "Lat": Lat,
    'Lng': Lng,
    'Max Temp': Max_Temp,
    'Humidity': Humidity,
    'Cloudiness': Cloudiness,
    'Wind Speed': Wind_Speed,
    'Country': Country,
    'Date': Date
    })
    
cities_df.to_csv('output')

cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,east london,-33.0153,27.9116,291.75,89.0,100.0,6.32,ZA,1.668814e+09
1,hilo,19.7297,-155.0900,300.51,82.0,75.0,4.63,US,1.668814e+09
2,ribeira grande,38.5167,-28.7000,291.91,68.0,95.0,7.15,PT,1.668814e+09
3,rikitea,-23.1203,-134.9692,295.10,73.0,11.0,5.47,PF,1.668814e+09
4,punta arenas,-53.1500,-70.9167,289.21,31.0,40.0,10.29,CL,1.668814e+09


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [15]:
#This step i skipped because the max Humidity is 
cities_df['Humidity'].max()

98.0

In [7]:
#  Get the indices of cities that have humidity over 100%.


In [8]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression